In [ ]:
# screen_recorder.py: screen recorder for video

import threading, os, time, sys
from PIL import ImageGrab
from moviepy.editor import ImageSequenceClip

fileName = input('FileName: ') or 'demo'
if not os.path.isdir(fileName):
    os.mkdir(fileName)
recording, imageList, recordTime = True, [], 0
def get_pictures():
    global recordTime
    t = time.time()
    while recording:
        image = ImageGrab.grab()
        imageName = os.path.join(
            fileName, '%s.jpg' % int(time.time() * 1e3))
        image.save(imageName)
        imageList.append(imageName)
        recordTime = time.time() - t
t = threading.Thread(target=get_pictures, daemon=True)
t.start()
print('Recording screen to %s.mp4, press Ctrl-C to stop' % fileName)
try:
    while 1:
        input()
except:
    recording = False
clip = ImageSequenceClip(imageList,
    fps=int(len(imageList) / recordTime))
clip.write_videofile('%s.mp4' % fileName)



In [ ]:
# test_video_recorder.py: for screen video record test

import json
import os
import shutil
import tempfile
import numpy as np

import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder


class BrokenRecordableEnv(object):
    metadata = {'render.modes': [None, 'rgb_array']}

    def render(self, mode=None):
        pass

class UnrecordableEnv(object):
    metadata = {'render.modes': [None]}

    def render(self, mode=None):
        pass

def test_record_simple():
    env = gym.make("CartPole-v1")
    rec = VideoRecorder(env)
    env.reset()
    rec.capture_frame()
    rec.close()
    assert not rec.empty
    assert not rec.broken
    assert os.path.exists(rec.path)
    f = open(rec.path)
    assert os.fstat(f.fileno()).st_size > 100

def test_no_frames():
    env = BrokenRecordableEnv()
    rec = VideoRecorder(env)
    rec.close()
    assert rec.empty
    assert rec.functional
    assert not os.path.exists(rec.path)

def test_record_unrecordable_method():
    env = UnrecordableEnv()
    rec = VideoRecorder(env)
    assert not rec.enabled
    rec.close()

def test_record_breaking_render_method():
    env = BrokenRecordableEnv()
    rec = VideoRecorder(env)
    rec.capture_frame()
    rec.close()
    assert rec.empty
    assert rec.broken
    assert not os.path.exists(rec.path)

def test_text_envs():
    env = gym.make('FrozenLake-v0')
    video = VideoRecorder(env)
    try:
        env.reset()
        video.capture_frame()
        video.close()
    finally:
        os.remove(video.path)

